In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
%run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_syllable_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', other_features=True)

data = Data('s')
data.generate_data('shuffled_syllable_reversed_train',
                    'shuffled_syllable_reversed_test',
                    'shuffled_syllable_reversed_validate')
#data.generate_data('TEST_train',
#                   'TEST_test',
#                   'TEST_validate')

LOADING DATA...
LOAD SUCCESSFUL!


In [4]:
%run ../../../prepare_data.py
# gen1 = generate_fake_epoch_syllables(X_train[:20], X_other_features_train[:20], y_train[:20], 16, 5168)
# gen2 = generate_fake_epoch_syllables(data.x_train[:20], data.x_other_features_train[:20], data.y_train[:20], 16, 5168)
gen2 = data.generator('train', 16)
# test1 = next(gen1)
test2 = next(gen2)

In [7]:
# prints test word from generator
# print(X_train[0])
# print(syllable_dictionary[test[0][0][0][0].tolist().index(1)])
pos = 0
# print(np.array(test1[0][0]).shape)
# for el in test1[0][0][pos]:
#     print(el)
#     print(syllable_dictionary[el.tolist().index(1)])
# print('-----------------------------------------------')
print(np.array(test2[0][0]).shape)
for el in test2[0][0][pos]:
    print(el)
    print(syllable_dictionary[el.tolist().index(1)])
    
print(data.decode_y(data.y_train[pos]))
print(data.y_train[pos])

(16, 10, 5168)
[0 0 0 ..., 0 0 0]
ša
[0 0 0 ..., 0 0 0]
lnej
[0 0 1 ..., 0 0 0]
a
[0 0 0 ..., 0 0 0]
ktu
[0 0 0 ..., 0 0 0]
ja
[0 0 0 ..., 0 0 0]
na
[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[2, 5]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]


In [6]:
num_examples = len(data.x_train) # training set size

nn_output_dim = 10
nn_hdim = 516
batch_size = 16
actual_epoch = 20
num_fake_epoch = 20



conv_input_shape=(10, 5168)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(160, (2), padding='same', activation='relu')(conv_input)
# x_conv = Conv1D(44, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)





model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [7]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=data.x_test.shape[0]/(batch_size * num_fake_epoch))

Epoch 1/400
1345/1344 [==============================] - 207s - loss: 0.2045 - actual_accuracy: 0.4271 - val_loss: 0.1298 - val_actual_accuracy: 0.6902
Epoch 2/400
1345/1344 [==============================] - 232s - loss: 0.1239 - actual_accuracy: 0.6692 - val_loss: 0.1046 - val_actual_accuracy: 0.7427
Epoch 3/400
1345/1344 [==============================] - 227s - loss: 0.0988 - actual_accuracy: 0.7326 - val_loss: 0.0922 - val_actual_accuracy: 0.7807
Epoch 4/400
1345/1344 [==============================] - 242s - loss: 0.0806 - actual_accuracy: 0.7902 - val_loss: 0.0836 - val_actual_accuracy: 0.8125
Epoch 5/400
1345/1344 [==============================] - 261s - loss: 0.0687 - actual_accuracy: 0.8260 - val_loss: 0.0838 - val_actual_accuracy: 0.8275
Epoch 6/400
1345/1344 [==============================] - 261s - loss: 0.0592 - actual_accuracy: 0.8511 - val_loss: 0.0796 - val_actual_accuracy: 0.8332
Epoch 7/400
1345/1344 [==============================] - 266s - loss: 0.0521 - actual_ac

1345/1344 [==============================] - 334s - loss: 0.0089 - actual_accuracy: 0.9804 - val_loss: 0.1427 - val_actual_accuracy: 0.8758
Epoch 55/400
1345/1344 [==============================] - 332s - loss: 0.0084 - actual_accuracy: 0.9819 - val_loss: 0.1295 - val_actual_accuracy: 0.8827
Epoch 56/400
1345/1344 [==============================] - 339s - loss: 0.0088 - actual_accuracy: 0.9806 - val_loss: 0.1522 - val_actual_accuracy: 0.8704
Epoch 57/400
1345/1344 [==============================] - 342s - loss: 0.0074 - actual_accuracy: 0.9841 - val_loss: 0.1427 - val_actual_accuracy: 0.8739
Epoch 58/400
1345/1344 [==============================] - 338s - loss: 0.0074 - actual_accuracy: 0.9845 - val_loss: 0.1509 - val_actual_accuracy: 0.8696
Epoch 59/400
1345/1344 [==============================] - 339s - loss: 0.0074 - actual_accuracy: 0.9844 - val_loss: 0.1466 - val_actual_accuracy: 0.8700
Epoch 60/400
1345/1344 [==============================] - 341s - loss: 0.0076 - actual_accuracy

1345/1344 [==============================] - 382s - loss: 0.0040 - actual_accuracy: 0.9921 - val_loss: 0.1790 - val_actual_accuracy: 0.8769
Epoch 108/400
1345/1344 [==============================] - 388s - loss: 0.0039 - actual_accuracy: 0.9918 - val_loss: 0.1801 - val_actual_accuracy: 0.8735
Epoch 109/400
1345/1344 [==============================] - 382s - loss: 0.0038 - actual_accuracy: 0.9923 - val_loss: 0.1802 - val_actual_accuracy: 0.8731
Epoch 110/400
1345/1344 [==============================] - 380s - loss: 0.0044 - actual_accuracy: 0.9912 - val_loss: 0.1678 - val_actual_accuracy: 0.8873
Epoch 111/400
1345/1344 [==============================] - 387s - loss: 0.0037 - actual_accuracy: 0.9921 - val_loss: 0.1692 - val_actual_accuracy: 0.8808
Epoch 112/400
1345/1344 [==============================] - 384s - loss: 0.0034 - actual_accuracy: 0.9932 - val_loss: 0.1879 - val_actual_accuracy: 0.8800
Epoch 113/400
1345/1344 [==============================] - 386s - loss: 0.0035 - actual_ac

1345/1344 [==============================] - 401s - loss: 0.0027 - actual_accuracy: 0.9947 - val_loss: 0.1794 - val_actual_accuracy: 0.8800
Epoch 161/400
1345/1344 [==============================] - 410s - loss: 0.0024 - actual_accuracy: 0.9957 - val_loss: 0.2321 - val_actual_accuracy: 0.8574
Epoch 162/400
1345/1344 [==============================] - 399s - loss: 0.0029 - actual_accuracy: 0.9943 - val_loss: 0.2067 - val_actual_accuracy: 0.8731
Epoch 163/400
1345/1344 [==============================] - 410s - loss: 0.0024 - actual_accuracy: 0.9950 - val_loss: 0.1798 - val_actual_accuracy: 0.8735
Epoch 164/400
1345/1344 [==============================] - 404s - loss: 0.0022 - actual_accuracy: 0.9959 - val_loss: 0.1835 - val_actual_accuracy: 0.8827
Epoch 165/400
1345/1344 [==============================] - 403s - loss: 0.0024 - actual_accuracy: 0.9954 - val_loss: 0.1994 - val_actual_accuracy: 0.8808
Epoch 166/400
1345/1344 [==============================] - 408s - loss: 0.0024 - actual_ac

1345/1344 [==============================] - 429s - loss: 0.0015 - actual_accuracy: 0.9970 - val_loss: 0.1883 - val_actual_accuracy: 0.8808
Epoch 214/400
1345/1344 [==============================] - 429s - loss: 0.0022 - actual_accuracy: 0.9957 - val_loss: 0.1908 - val_actual_accuracy: 0.8869
Epoch 215/400
1345/1344 [==============================] - 414s - loss: 0.0019 - actual_accuracy: 0.9961 - val_loss: 0.1897 - val_actual_accuracy: 0.8850
Epoch 216/400
1345/1344 [==============================] - 416s - loss: 0.0016 - actual_accuracy: 0.9966 - val_loss: 0.1908 - val_actual_accuracy: 0.8880
Epoch 217/400
1345/1344 [==============================] - 416s - loss: 0.0017 - actual_accuracy: 0.9966 - val_loss: 0.2065 - val_actual_accuracy: 0.8788
Epoch 218/400
1345/1344 [==============================] - 413s - loss: 0.0019 - actual_accuracy: 0.9960 - val_loss: 0.2104 - val_actual_accuracy: 0.8742
Epoch 219/400
1345/1344 [==============================] - 424s - loss: 0.0018 - actual_ac

1345/1344 [==============================] - 438s - loss: 0.0015 - actual_accuracy: 0.9976 - val_loss: 0.2087 - val_actual_accuracy: 0.8738
Epoch 267/400
1345/1344 [==============================] - 440s - loss: 0.0014 - actual_accuracy: 0.9970 - val_loss: 0.2299 - val_actual_accuracy: 0.8738
Epoch 268/400
1345/1344 [==============================] - 435s - loss: 0.0014 - actual_accuracy: 0.9972 - val_loss: 0.1847 - val_actual_accuracy: 0.8877
Epoch 269/400
1345/1344 [==============================] - 439s - loss: 0.0012 - actual_accuracy: 0.9978 - val_loss: 0.2140 - val_actual_accuracy: 0.8735
Epoch 270/400
1345/1344 [==============================] - 433s - loss: 0.0016 - actual_accuracy: 0.9971 - val_loss: 0.2133 - val_actual_accuracy: 0.8742
Epoch 271/400
1345/1344 [==============================] - 445s - loss: 0.0016 - actual_accuracy: 0.9970 - val_loss: 0.2038 - val_actual_accuracy: 0.8815
Epoch 272/400
1345/1344 [==============================] - 443s - loss: 0.0015 - actual_ac

1345/1344 [==============================] - 453s - loss: 0.0010 - actual_accuracy: 0.9977 - val_loss: 0.2215 - val_actual_accuracy: 0.8762
Epoch 320/400
1345/1344 [==============================] - 451s - loss: 0.0013 - actual_accuracy: 0.9972 - val_loss: 0.2251 - val_actual_accuracy: 0.8762
Epoch 321/400
1345/1344 [==============================] - 454s - loss: 0.0012 - actual_accuracy: 0.9977 - val_loss: 0.2469 - val_actual_accuracy: 0.8590
Epoch 322/400
1345/1344 [==============================] - 453s - loss: 0.0018 - actual_accuracy: 0.9967 - val_loss: 0.2368 - val_actual_accuracy: 0.8662
Epoch 323/400
1345/1344 [==============================] - 454s - loss: 9.1612e-04 - actual_accuracy: 0.9980 - val_loss: 0.2261 - val_actual_accuracy: 0.8785
Epoch 324/400
1345/1344 [==============================] - 446s - loss: 0.0013 - actual_accuracy: 0.9974 - val_loss: 0.2002 - val_actual_accuracy: 0.8792
Epoch 325/400
1345/1344 [==============================] - 451s - loss: 0.0013 - actua

1345/1344 [==============================] - 445s - loss: 0.0013 - actual_accuracy: 0.9973 - val_loss: 0.2483 - val_actual_accuracy: 0.8643
Epoch 373/400
1345/1344 [==============================] - 453s - loss: 0.0012 - actual_accuracy: 0.9977 - val_loss: 0.2338 - val_actual_accuracy: 0.8708
Epoch 374/400
1345/1344 [==============================] - 447s - loss: 0.0011 - actual_accuracy: 0.9983 - val_loss: 0.1998 - val_actual_accuracy: 0.8911
Epoch 375/400
1345/1344 [==============================] - 449s - loss: 0.0016 - actual_accuracy: 0.9966 - val_loss: 0.2006 - val_actual_accuracy: 0.8746
Epoch 376/400
1345/1344 [==============================] - 455s - loss: 9.8074e-04 - actual_accuracy: 0.9978 - val_loss: 0.2354 - val_actual_accuracy: 0.8600
Epoch 377/400
1345/1344 [==============================] - 458s - loss: 0.0013 - actual_accuracy: 0.9972 - val_loss: 0.2123 - val_actual_accuracy: 0.8812
Epoch 378/400
1345/1344 [==============================] - 453s - loss: 0.0014 - actua

In [18]:
name = '20_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [6]:
# dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
# train_content, test_content, validate_content = split_content(content, 0.2, '../../internal_representations/inputs/content_shuffle_vector.h5')
# feature_dictionary = create_feature_dictionary()
# syllable_dictionary = create_syllables_dictionary(content, vowels)


content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)